<a href="https://colab.research.google.com/github/vitroid/ClustIce/blob/tilecycles/clustice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install py3Dmol

In [ ]:
pip install git+https://github.com/vitroid/ClustIce.git@tilecycles

In [1]:
from logging import getLogger, INFO, DEBUG, basicConfig

import networkx as nx
import py3Dmol

from clustice.geometry import make_layout
from clustice.gromacs import render
from clustice.topology import ice_graph
from clustice.coder import encode
from clustice.water import tip4p
from clustice import graph


logger = getLogger()
basicConfig(level=DEBUG)

# note: g must be a graph whose labels start from 0.
# g = graph.great_icosahedron(2)
g = graph.great_decahedron(2)
# g = graph.small_barrelan()
# g = graph.large_barrelan()
# g = graph.twistane()
# g = graph.adamantane()
# g = nx.cycle_graph(6) # hexagon
# g = nx.cycle_graph(7) # heptagon
# g = nx.cubical_graph() # cubic octamer
# g = nx.dodecahedral_graph() # cubic octamer


# O-O distance
L = 0.27

# estimate of the positions of the nodes
layout = make_layout(g, edgelen=L)

# set orientations of the hydrogen bonds.
# if pos is given, the net dipole moment is minimized.
dg = ice_graph(g, pos=layout)
# dg = ice_graph(g)

# get the unique id for the graph
id = encode(dg)
print(id)

# put water molecules
gro = render(dg, layout, watermodel=tip4p)
with open('save.gro','w') as f:
    f.write(gro)

# show
view = py3Dmol.view()
view.addModel(gro, 'gro')
view.setStyle({'stick':{}})
view.addUnitCell()
view.zoomTo()
view.show()


/Users/matto/.local/share/virtualenvs/ClustIce-TrkXxxIw/lib/python3.11/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(
INFO:root:1 2.1228743168029105 RG
INFO:root:2 1.9676036757499051 RG
INFO:root:4 1.7431408308310843 RG
INFO:root:8 1.4829774246092335 RG
INFO:root:16 1.2682022747121513 RG
INFO:root:32 1.154386545435404 RG
INFO:root:64 1.1132361027666604 RG
DEBUG:root:decomposing [0, 35, 1, 221, 166, 201, 167, 112, 147, 117, 182, 202, 172, 206, 176]...
DEBUG:root:[{0: 0, 35: 1}, [0, 35]]
DEBUG:root:[{0: 0, 35: 1, 1: 2}, [0, 35, 1]]
DEBUG:root:[{0: 0, 35: 1, 1: 2, 221: 3}, [0, 35, 1, 221]]
DEBUG:root:[{0: 0, 35: 1, 1: 2, 221: 3, 166: 4}, [0, 35, 1, 221, 166]]
DEBUG:root:[{0: 0, 35: 1, 1: 2, 221: 3, 166: 4, 201: 5}, [0, 35, 1, 221, 166, 201]]
DEBUG:root:[{0: 0, 35: 1, 1: 2, 221: 3, 166: 4, 201: 5, 167: 6}, [0,

35+220.36+221.57+222.58+223.59.35+70.39+40.37+72.38.39.42+43.41+82.86.43+44.89.225.36+45.37+46.38+47.48.40+46.41+49.42.49+50.44.45+229.51+230.47+231.48.52+53.50.51+232.52+54.53.54.1+15.2+6.2+3.3+4.5+19.7+10.7+8.9+12.11+23.12+14.15+19.16+26.17+21.20+22.20+29.22+24.25+28.26+27.28+30.31+33.0.1+91.91+112.92+93.93.90+125.95+126.92+127.96+128.94.95+98.96.97+99.99..90+100.6+100.101+102.8.94.101+103.102+104.97+105.98+104.105.9.10+107.107.103+106.108.108.106.13+109..109.55+56.61+71.57+72.63+73.60+61.62+75.62+76.64+65.66+68.69+79.74+80.71+81.73+82.74+77.75+84.78+85.80+81.84+87.83+88.86+88.55.56+145.146+147.58+168.59+169.149.149+181.150+182.148+151.152+190.150+153.153+192.196.154+197.154+199.145.146+155.147+157.148+157.155+158.156+159.151.152+158.159+160..64+161.65+156.66+162.161+163.162.160+163.67.68.164.69+164.110+115.111+116.113+117.113+114.119+129.116+130.117+121.120+122.123+133.122+134.125+135.126+127.131+137.130+138.131+139.132+134.136+141.136+142.139+143.141+142.110.111+201.112+202.203+223

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Rebuild the cluster from id.

In [ ]:
import networkx as nx

from clustice.geometry import make_layout
from clustice.gromacs import render
from clustice.coder import decode
from clustice.water import tip4p

import py3Dmol

s = "1.2.3+6.19.3.4+6.7.14.1+7.8+13.0+9.10+12.16.12.13+15.5.15+17.4+18.11.0+18"

dg = decode(s)

# Make layout of the nodes
g = nx.Graph(dg)
L = 0.27
layout = make_layout(g, edgelen=L)

# put water molecules
gro = render(dg, layout, watermodel=tip4p)

# show
view = py3Dmol.view()
view.addModel(gro, 'gro')
view.setStyle({'stick':{}})
view.addUnitCell()
view.zoomTo()
view.show()


In [ ]:
pip install GenIce2

In [ ]:
import networkx as nx
from genice2.genice import GenIce
from genice2.plugin import Format, Lattice

from clustice.gromacs import render
from clustice.topology import ice_graph
from clustice.water import tip4p

import py3Dmol

lattice = Lattice("1h")
formatter = Format("raw", stage=(1, 2))
raw = GenIce(lattice, signature="Ice Ih", rep=(3, 3, 3)).generate_ice(
    formatter
)

# graph is the topology of the hydrogen-bond network
g = nx.Graph(raw["graph"])
# reppositions contains the positions of CoM of water in fractional coordinate
layout = raw["reppositions"]
# repcell is the cell matrix (transposed)
cell = raw["repcell"]

# set orientations of the hydrogen bonds.
# It automatically depolarizes by optimizing the arrangement.
dg = ice_graph(g, pos=layout, pbc=True)

# put water molecules
gro = render(
    dg,
    layout @ cell,
    watermodel=tip4p,
    cell=f"{cell[0,0]} {cell[1,1]} {cell[2,2]}",
    pbc=True,
)

# show
view = py3Dmol.view()
view.addModel(gro, 'gro')
view.setStyle({'stick':{}})
view.addUnitCell()
view.zoomTo()
view.show()


# Benchmark tests

In [ ]:
import networkx as nx

from clustice.geometry import make_layout
from clustice.gromacs import render
from clustice.topology import ice_graph
from clustice.coder import encode
from clustice.water import tip4p
from clustice.graph import great_icosahedron

import py3Dmol
from logging import getLogger, INFO

logger = getLogger()
logger.setLevel(INFO)

# O-O distance
L = 0.27

# note: g must be a graph whose labels start from 0.
# g = nx.dodecahedral_graph() # dodecahedral 20mer
g = great_icosahedron(3)
layout = make_layout(g, edgelen=L)
print(len(layout))


In [ ]:
%%timeit

# estimate of the positions of the nodes
layout = make_layout(g, edgelen=L)



In [ ]:
%%timeit
dg = ice_graph(g, pos=layout)



In [ ]:
%%timeit
dg = ice_graph(g)


In [ ]:
# great_icosahedron
# size  nnode   constellation        depolarization       ice rule
benchmark=[
    [0, 20,  0.146, 0.00612, 0.000410],
    [1, 100, 0.676, 0.00865, 0.00229],
    [2, 280, 3.14,  0.0178,  0.00569],
    [3, 600, 51.4,  0.024,   0.0133]